### Dining Philosophers with Message Passing (Go)

A deadlock among philosophers is prevented if every philosopher picks up their lower-numbered fork first. The following program has a *runtime monitor* to check that, and the philosophers are *instrumented* with statements communicating their actions to the runtime monitor. The implementation of the philosophers is flawed as every philosopher picks up their left-hand fork (through that fork's right-hand channel) and then their right-hand fork (through that fork's left-hand channel). Hence, the runtime monitor will, at some point, flag that situation and terminate the program.

In [1]:
%%writefile philosophers.go
package main

import ("time"; "math/rand")

type PhilFork struct{ph int; f int}

var left, right [5]chan bool
var ph [5]string
var log chan PhilFork

func philosopherState(i int, s string) {
    ph[i] = s; println(ph[0], ph[1], ph[2], ph[3], ph[4])
    time.Sleep(time.Second * time.Duration(rand.Int() % 3)) // eating or thinking 0..2 sec
}
func fork(f int) {
    for {
        select {
            case <- left[f]: <- left[f]
            case <- right[f]: <- right[f]
        }
    }
}
func runtimemonitor() {
    ff := []int{-1, -1, -1, -1, -1} // first picked up fork; -1 if no fork picked up
    for p := range log {
        if ff[p.ph] == -1 {ff[p.ph] = p.f//; println(p.ph, "picking up", p.f, "first")
        } else {//println(p.ph, "picking up", p.f, "second")
            if ff[p.ph] < p.f {ff[p.ph] = -1
            } else {panic("higher-numbered fork picked first")
            }
        }
    }
}
func philosopher(ph int) {
    for {
        l, r := ph, (ph + 1) % 5
        left[l] <- true; log <- PhilFork{ph, l}; right[r] <- true; log <- PhilFork{ph, r}
        philosopherState(ph, "eats  ")
        left[l] <- false; right[r] <- false
        philosopherState(ph, "thinks")
    }
}
func main() {
    log = make(chan PhilFork)
    go runtimemonitor()
    for i := 0; i < 5; i++ {left[i], right[i], ph[i] = make(chan bool), make(chan bool), "thinks"}
    for i := 0; i < 5; i++ {go fork(i); go philosopher(i)}
    time.Sleep(20 * time.Second)
}

Writing philosophers.go


In [2]:
!go run philosophers.go

thinks thinks thinks thinks eats  
panic: higher-numbered fork picked first

goroutine 5 [running]:
main.runtimemonitor()
	/home/ahmem73/Assignment10/philosophers.go:29 +0xc5
created by main.main in goroutine 1
	/home/ahmem73/Assignment10/philosophers.go:45 +0x54
exit status 2


Modify it such that, instead of picking up the left and then the right fork, philosophers pick up the lower-numbered fork and then the higher-numbered fork! After `left[l] <- true` add `log <- PhilFork{ph, l}` and after `right[r] <- true` add `log <- PhilFork{ph, r}`, like above. [2 points]

In [1]:
%%writefile philosophers.go
package main

import ("time"; "math/rand")

type PhilFork struct{ph int; f int}

var left, right [5]chan bool
var ph [5]string
var log chan PhilFork

func philosopherState(i int, s string) {
    ph[i] = s; println(ph[0], ph[1], ph[2], ph[3], ph[4])
    time.Sleep(time.Second * time.Duration(rand.Int() % 3)) // eating or thinking 0..2 sec
}
func fork(f int) {
    for {
        select {
            case <- left[f]: <- left[f]
            case <- right[f]: <- right[f]
        }
    }
}
func runtimemonitor() {
    ff := []int{-1, -1, -1, -1, -1} // first picked up fork; -1 if no fork picked up
    for p := range log {
        if ff[p.ph] == -1 {ff[p.ph] = p.f//; println(p.ph, "picking up", p.f, "first")
        } else {//println(p.ph, "picking up", p.f, "second")
            if ff[p.ph] < p.f {ff[p.ph] = -1
            } else {panic("higher-numbered fork picked first")
            }
        }
    }
}
func philosopher(ph int) {
    for {
        l, r := ph, (ph+1)%5
        if l < r {
            left[l] <- true; log <- PhilFork{ph,l}
            right[r] <- true; log <- PhilFork{ph,r}
        } else {
            right[r] <- true; log <- PhilFork{ph,r}
            left[l] <- true; log <- PhilFork{ph,l}
        }
        philosopherState(ph, "eats  ")
        left[l] <- false; right[r] <- false
        philosopherState(ph, "thinks")
    }
}
func main() {
    log = make(chan PhilFork)
    go runtimemonitor()
    for i := 0; i < 5; i++ {left[i], right[i], ph[i] = make(chan bool), make(chan bool), "thinks"}
    for i := 0; i < 5; i++ {go fork(i); go philosopher(i)}
    time.Sleep(20 * time.Second)
}

Overwriting philosophers.go


In [2]:
!go run philosophers.go

thinks thinks thinks thinks eats  
thinks thinks thinks thinks thinks
thinks thinks thinks eats   thinks
thinks thinks thinks thinks thinks
thinks thinks eats   thinks thinks
thinks thinks thinks thinks thinks
thinks thinks eats   thinks thinks
thinks thinks thinks thinks thinks
thinks thinks thinks eats   thinks
thinks eats   thinks thinks thinks
thinks eats   thinks eats   thinks
thinks thinks thinks thinks thinks
thinks thinks eats   thinks thinks
eats   thinks eats   thinks thinks
thinks thinks eats   thinks thinks
thinks thinks eats   thinks eats  
thinks thinks eats   thinks thinks
thinks thinks eats   thinks eats  
thinks thinks eats   thinks thinks
thinks thinks thinks thinks thinks
thinks eats   thinks thinks thinks
thinks eats   thinks eats   thinks
eats   thinks thinks eats   thinks
thinks thinks thinks eats   thinks
thinks thinks thinks eats   thinks
thinks thinks eats   thinks thinks
thinks thinks thinks thinks thinks
thinks thinks thinks thinks eats  
thinks thinks eats  

Realizing their eternal dilemma, the Dining Philosophers decide to hire a footman whose task to allow only four philosophers to sit on the table; the philosophers agree to think in the library. When entering and exiting the dining hall, the philosophers have to check with the footman. Complete `footman` below! [4 points]

In [7]:
%%writefile philosophers.go
package main

import ("time"; "math/rand")

var left, right [5]chan bool
var ph [5]string
var enter, exit chan bool
var log chan string

func footman() {
    occ := 0
    for {
        if occ<4 {
            select{
                case <- enter:
                    occ += 1
                case <- exit:
                    occ -= 1
            }
        } else {
            <- exit
            occ -= 1
        }
    }
}
func runtimemonitor() {
    sitting := 0
    for m := range log {
        if m == "exit" {
            sitting -= 1
        } else {sitting += 1; if sitting == 5 {panic("5 philosophers sitting")}
        }
    }
}
func philosopherState(i int, s string) {
    ph[i] = s; println(ph[0], ph[1], ph[2], ph[3], ph[4])
    time.Sleep(time.Second * time.Duration(rand.Int() % 3)) // eating or thinking 0..2 sec
}
func fork(i int) {
    for {
        select {
            case <-left[i]: <- left[i]
            case <-right[i]: <- right[i]
        }
    }
}
func philosopher(i int) {
    for {
        enter <- true; log <- "enter"
        left[i] <- true; right[(i + 1) % 5] <- true
        philosopherState(i, "eats  ")
        left[i] <- false; right[(i + 1) % 5] <- false
        philosopherState(i, "thinks")
        log <- "exit"; 
        exit <- true
    }
}
func main() {
    for i := 0; i < 5; i++ {left[i], right[i], ph[i] = make(chan bool), make(chan bool), "thinks"}
    enter, exit, log = make(chan bool), make(chan bool), make(chan string)
    go footman(); go runtimemonitor()
    for i := 0; i < 5; i++ {go fork(i); go philosopher(i)}
    time.Sleep(20 * time.Second)
}

Overwriting philosophers.go


In [8]:
!go run philosophers.go

thinks thinks thinks thinks eats  
thinks thinks eats   thinks eats  
thinks eats   eats   thinks eats  
thinks eats   thinks thinks eats  
thinks eats   thinks thinks thinks
thinks eats   thinks eats   thinks
eats   thinks thinks eats   thinks
thinks thinks thinks eats   thinks
thinks thinks thinks eats   thinks
thinks thinks thinks thinks thinks
thinks thinks eats   thinks thinks
thinks thinks eats   thinks eats  
thinks thinks eats   thinks thinks
thinks thinks thinks thinks thinks
thinks eats   thinks thinks thinks
thinks eats   thinks eats   thinks
thinks eats   thinks thinks thinks
thinks thinks thinks thinks thinks
eats   thinks thinks thinks thinks
thinks thinks thinks thinks thinks
thinks thinks thinks thinks eats  
thinks thinks eats   thinks thinks
eats   thinks eats   thinks thinks
thinks thinks eats   thinks eats  
thinks thinks eats   thinks thinks
thinks thinks eats   thinks eats  
thinks thinks thinks thinks eats  
thinks thinks thinks thinks thinks
thinks thinks thinks

Above code is instrumented for monitoring that not more than 4 philosophers are sitting at the table. Is the monitor *sound*, meaning it only reports if there are indeed 5 philosophers at the table, and is it *complete*, meaning that it never misses a state in which there are 5 philosophers? [1 point]

The monitor is sound since it makes sure no more than 4 can sit at the table and it also raises a panic if more than 4 are sitting at the table. the monitor isnt complete since the asynch nature of channel communication there can be race conditions that could occur. due to this the monitor could miss a state where there is infact 5 philosophers at the table

Running any of above versions with runtime race detection of Go enabled should detect a race condition. Where does it originate and is it critical for the correctness of the program? [1 point]

In [ ]:
!go run -race philosophers.go

the race conditions detected originate from the unsynch shared access of the ph array which is being modified by multiple goroutines without any synch mechanisms in place. The main purpose of this array is for logging so it does not effect the correctness of the program